In [6]:
import sys
!conda install --yes --prefix {sys.prefix} matplotlib

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
# Script to get JSON data from 
import re
import matplotlib.pyplot as plt
import json
import os
#cwd = os.getcwd()
cwd = "C:\\Users\\maria\\Documents\\Grad School\\Fall2018\\CMSC611\\Projects\\all-20181213T021317Z-001\\all\\"
stats_dir = "spark-stats"
base_dir = os.path.dirname(cwd)

metric_output_mapping = {
    "basic-dataframe-run-output": "BasicDataframe",
    "basic-dataset-run-output": "BasicDataset",
    "basic-rdd-run-output": "BasicRDD",
    "cache-dataframe-run-output": "CacheDataframe",
    "cache-dataset-run-output": "CacheDataset",
    "cache-rdd-run-output": "CacheRDD",
    "kmeans-dataframe-run-output": "KMeansDataframe",
    "kmeans-dataset-run-output": "KMeansDataset",
    "map-partitions-rdd-run-output": "MapPartitionsRDD",
    "map-rdd-run-output": "MapRDD",
    "partition-dataframe-run-output": "PartitionDataframe",
    "parititon-dataset-run-output": "PartitionDataset",
    "partition-rdd-run-output": "PartitionRDD"
}

def get_associated_files(run_output_name):
    files = os.listdir(base_dir + "\\" + stats_dir)
    return [a_file for a_file in files if a_file.startswith(metric_output_mapping[run_output_name])]

print(cwd)
print(base_dir)
print(get_associated_files("basic-dataframe-run-output"))

C:\Users\maria\Documents\Grad School\Fall2018\CMSC611\Projects\all-20181213T021317Z-001\all\
C:\Users\maria\Documents\Grad School\Fall2018\CMSC611\Projects\all-20181213T021317Z-001\all
['BasicDataframeRunner-2018-12-09T07_07_23.1544339243Z', 'BasicDataframeRunner-2018-12-09T07_09_59.1544339399Z', 'BasicDataframeRunner-2018-12-09T07_12_34.1544339554Z', 'BasicDataframeRunner-2018-12-09T07_15_07.1544339707Z', 'BasicDataframeRunner-2018-12-09T07_17_42.1544339862Z', 'BasicDataframeRunner-2018-12-09T07_19_10.1544339950Z', 'BasicDataframeRunner-2018-12-09T07_20_38.1544340038Z', 'BasicDataframeRunner-2018-12-09T07_22_07.1544340127Z', 'BasicDataframeRunner-2018-12-09T07_23_36.1544340216Z', 'BasicDataframeRunner-2018-12-09T07_25_05.1544340305Z', 'BasicDataframeRunner-2018-12-09T07_26_13.1544340373Z', 'BasicDataframeRunner-2018-12-09T07_27_23.1544340443Z', 'BasicDataframeRunner-2018-12-09T07_28_33.1544340513Z', 'BasicDataframeRunner-2018-12-09T07_29_42.1544340582Z', 'BasicDataframeRunner-2018-12-

In [ ]:
# Graph to compare min and max finish time amongst all of the metrics
% matplotlib inline
import matplotlib.pyplot as plt
import math
import json
import os

# Extracts statistics from the data object which is an array of job dictionaries
# processes stat if necessary
def extract_stat(data_obj, stat_x, stat_y):
    stats_x = []
    stats_y = []
    for job in data_obj:
        stat_value_x = None
        stat_value_y = None
        # Account for stats in which two values need to be subtracted
        if isinstance(stat_x, tuple):
            stat_value_x = abs(job[stat_x[0]] - job[stat_x[1]])
        if isinstance(stat_y, tuple):
            stat_value_y = abs(job[stat_y[0]] - job[stat_y[1]])
        if not stat_value_x:
            stat_value_x = job[stat_x]
        if not stat_value_y:
            stat_value_y = job[stat_y]
            
        stats_x.append(stat_value_x)
        stats_y.append(stat_value_y)
    return (max(stats_x), max(stats_y))

# generates a graph that compares the given metrics with the 2 statistics
# 
# metrics - The list of metrics
# stat_x - The name of the stat as shown in the json. If the stat contains a tuple of 2, the difference
# of the first and second in the tuple will be used.
# stat_y - The name of the other stat as shown in the json
def generate_comparison(metrics, stat_x, stat_y):

    n_groups=16
    fig, ax = plt.subplots(figsize=(15,10))

    index = np.arange(n_groups)
    bar_width = 0.2

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    metric_stats = {}
    for metric in metrics:
        metric_stats[metric] = {stat_x: None, stat_y:None}
        data_files = get_associated_files(metric)
        stat_values_x = []
        stat_values_y = []
        for data_file in data_files:
            # Compile the data file path
            data_file_path = os.path.join(cwd, stats_dir)
            fp = open(pathed)
            data_obj = json.load(fp)
            stat_value_x, stat_value_y = extract_stat(data_obj, stat_x, stat_y)
            stat_values_x.append(stat_value_x)
            stat_values_y.append(stat_value_y)
        metric_stats[metric][stat_x] = max(stat_values_x)
        metric_stats[metric][stat_y] = max(stat_values_y)
        
    for # TODO, add a line for each metric instead of bar
        rects1 = ax.bar(x, [math.log(i,10) for i in y1], bar_width,
                        alpha=opacity,
                        error_kw=error_config,
                        label='Dataframe')

    ax.set_xlabel('Job Number (Determines Memory Setting)')
    ax.set_ylabel('Peak Memory (Log Base 10)')
    ax.set_title('Peak Execution Memory for Each Memory Setting')
    ax.set_xticks(1 + index + bar_width)
    ax.set_xticklabels(x)
    ax.legend()

    fig.tight_layout()
    plt.show()